In [1]:
import pandas as pd
import numpy as np
import os
import shapely
from shapely.geometry import Point, Polygon
import geopandas as gpd

print("done")


done


In [2]:
# Function to extract the numbers before the first underscore
def extract_numbers(value):
    return value.split('_')[0]

# List of files to process with full paths
files = [
    r'D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\may_meteo24.csv',
    r'D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\abr_meteo24.csv'
]

for file in files:
    # Read the CSV file
    df = pd.read_csv(file, sep=";")
    
    # Check if the column 'PUNTO_MUESTREO' exists in the DataFrame
    if 'PUNTO_MUESTREO' in df.columns:
        # Create the new column and fill it with the extracted values
        df['ID_EST'] = df['PUNTO_MUESTREO'].apply(extract_numbers)
        
        # Reorder columns to make the new column the first one
        columns_order = ['ID_EST'] + [col for col in df.columns if col != 'ID_EST']
        df = df[columns_order]
        #drop unnecessary cols
        df.drop(df.columns[[1, 2, 3]], axis=1, inplace=True)
        # Drop all validation columns 
        df = df.loc[:, ~df.columns.str.startswith('V')]
        # Reshape the DataFrame to have hourly data in rows
        df_melted = df.melt(id_vars=['ID_EST', 'MAGNITUD', 'PUNTO_MUESTREO', 'ANO', 'MES', 'DIA'],
                            var_name='HOUR', value_name='VALUE')
        
        # Extract the hour from the 'HOUR' column
        df_melted['HOUR'] = df_melted['HOUR'].str.extract('(\d+)', expand=False).astype(int)
        
        # Sort the DataFrame by ID_EST, MAGNITUD, and timestamp
        df_melted = df_melted.sort_values(by=['ID_EST', 'MAGNITUD', 'ANO', 'MES', 'DIA', 'HOUR'])
        # Save the modified DataFrame to a new CSV file
        output_file = file.replace('.csv', '_modified.csv')
        df_melted.to_csv(output_file, index=False)
        


In [3]:
#concatenate the modified files 
may = pd.read_csv('D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\may_meteo24_modified.csv',sep=",")
apr = pd.read_csv('D:\\Data projects\\Air pollution Madrid\\data\\meteo data\\abr_meteo24_modified.csv',sep=",")
meteo_data = pd.concat([apr, may], ignore_index=True)

In [4]:
# Now we merge the air_data with the coordinates of each station
# Load the CSV file
file_path = "D:\\Data projects\\Air pollution Madrid\\data\\locations\\meteo_locations_modified.csv"
locations_df = pd.read_csv(file_path, sep=",")


# Convert the ID_EST column to int to match the ID column in locations_df
meteo_data['ID_EST'] = meteo_data['ID_EST'].astype(int)

# Ensure the ID column in locations_df is also of type int (it usually should be)
locations_df['ID'] = locations_df['ID'].astype(int)

# Merge the DataFrames on the corrected ID columns
merged_df = pd.merge(meteo_data, locations_df, left_on='ID_EST', right_on='ID', how='left')

# Specify the directory and filename to save the combined DataFrame
output_directory = r"D:\\Data projects\\Air pollution Madrid\\data\\meteo data"
output_filename = 'meteo_geolocated.csv'
output_file = os.path.join(output_directory, output_filename)

# Save the combined DataFrame to a new CSV file
merged_df.to_csv(output_file, index=False)

print("This is the resulting dataset")
merged_df

This is the resulting dataset


,ID_EST,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,HOUR,VALUE,ID,LATITUD,LONGITUD
0,28079004,83,28079004_83_98,2024,4,1,1,6.2,28079004,404.238.823,-37.122.567
1,28079004,83,28079004_83_98,2024,4,1,2,6.1,28079004,404.238.823,-37.122.567
2,28079004,83,28079004_83_98,2024,4,1,3,5.7,28079004,404.238.823,-37.122.567
3,28079004,83,28079004_83_98,2024,4,1,4,5.4,28079004,404.238.823,-37.122.567
4,28079004,83,28079004_83_98,2024,4,1,5,5.4,28079004,404.238.823,-37.122.567
...,...,...,...,...,...,...,...,...,...,...,...
132883,28079115,86,28079115_86_98,2024,5,31,20,16.0,28079115,403.925.444,-3.697.631
132884,28079115,86,28079115_86_98,2024,5,31,21,18.0,28079115,403.925.444,-3.697.631
132885,28079115,86,28079115_86_98,2024,5,31,22,20.0,28079115,403.925.444,-3.697.631
132886,28079115,86,28079115_86_98,2024,5,31,23,23.0,28079115,403.925.444,-3.697.631


In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132888 entries, 0 to 132887
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID_EST          132888 non-null  int32  
 1   MAGNITUD        132888 non-null  int64  
 2   PUNTO_MUESTREO  132888 non-null  object 
 3   ANO             132888 non-null  int64  
 4   MES             132888 non-null  int64  
 5   DIA             132888 non-null  int64  
 6   HOUR            132888 non-null  int64  
 7   VALUE           132888 non-null  float64
 8   ID              132888 non-null  int32  
 9   LATITUD         132888 non-null  object 
 10  LONGITUD        132888 non-null  object 
dtypes: float64(1), int32(2), int64(5), object(3)
memory usage: 10.1+ MB


In [6]:
# now turn the magnitudes into variables 
# Mapping of MAGNITUD codes to parameter names
magnitude_mapping = {
    80: 'ULTRAVIOLETS',
    81: 'WIND_SPEED',
    82: 'WIND_DIR',
    83: 'TEMPERATURE',
    86: 'HUMIDITY_REL',
    87: 'PRESSURE',
    88: 'SOLAR_RAD',
    89: 'PRECIPITATION'
}

# Replace MAGNITUD codes with their respective names
merged_df['MAGNITUD'] = merged_df['MAGNITUD'].map(magnitude_mapping)

# Pivot the table
merged_df_pivot = merged_df.pivot_table(index=['ID_EST', 'ANO', 'MES', 'DIA', 'HOUR', 'LATITUD', 'LONGITUD'],
                                columns='MAGNITUD',
                                values='VALUE').reset_index()

# Flatten the columns
merged_df_pivot.columns.name = None

# Display the first few rows of the pivoted DataFrame
print(merged_df_pivot.head())

     ID_EST   ANO  MES  DIA  HOUR      LATITUD     LONGITUD  HUMIDITY_REL  \
0  28079004  2024    4    1     1  404.238.823  -37.122.567           NaN   
1  28079004  2024    4    1     2  404.238.823  -37.122.567           NaN   
2  28079004  2024    4    1     3  404.238.823  -37.122.567           NaN   
3  28079004  2024    4    1     4  404.238.823  -37.122.567           NaN   
4  28079004  2024    4    1     5  404.238.823  -37.122.567           NaN   

   PRECIPITATION  PRESSURE  SOLAR_RAD  TEMPERATURE  WIND_DIR  WIND_SPEED  
0            NaN       NaN        NaN          6.2       NaN         NaN  
1            NaN       NaN        NaN          6.1       NaN         NaN  
2            NaN       NaN        NaN          5.7       NaN         NaN  
3            NaN       NaN        NaN          5.4       NaN         NaN  
4            NaN       NaN        NaN          5.4       NaN         NaN  


In [7]:
# Display the first few rows of the meteo DataFrame before pivoting
print(merged_df.head())

# Pivot the table
meteo_pivot = merged_df.pivot_table(index=['ID_EST', 'ANO', 'MES', 'DIA', 'HORA', 'LATITUD', 'LONGITUD'],
                                columns='MAGNITUD',
                                values='VALUE', aggfunc='mean').reset_index()

# Flatten the columns
merged_df.columns.name = None

# Display the first few rows of the pivoted DataFrame
print(merged_df.head())

     ID_EST     MAGNITUD  PUNTO_MUESTREO   ANO  MES  DIA  HOUR  VALUE  \
0  28079004  TEMPERATURE  28079004_83_98  2024    4    1     1    6.2   
1  28079004  TEMPERATURE  28079004_83_98  2024    4    1     2    6.1   
2  28079004  TEMPERATURE  28079004_83_98  2024    4    1     3    5.7   
3  28079004  TEMPERATURE  28079004_83_98  2024    4    1     4    5.4   
4  28079004  TEMPERATURE  28079004_83_98  2024    4    1     5    5.4   

         ID      LATITUD     LONGITUD  
0  28079004  404.238.823  -37.122.567  
1  28079004  404.238.823  -37.122.567  
2  28079004  404.238.823  -37.122.567  
3  28079004  404.238.823  -37.122.567  
4  28079004  404.238.823  -37.122.567  


KeyError: 'HORA'